## Annotated CycleGAN

As we are building a web app "deepfake" which is of image translation based on GAN model, I present an "annotated" version of the pytorch-based-CycleGAN model in the form of a line-by-line implementation. It will cover data preparation, different GAN models, training schemes, optimization tricks for both training and computation. After completing this notebook, you will know:

- How to implement the discriminator and generator models.
- How to define composite models to train the two aforementioned models via adversarial and cycle loss.
- How to do the model inference to generate the translated image.

To follow along you will first need to install the libraries listed in `requirements-an.txt`. The workflow should be considered as a starting point of image translation tasks with the enrichment of GAN model.

> My comments are blockquoted. The main text is based on the CycleGAN/Pytorch documentation.
> Wangsu Hu (wangsu.hu1234@gmail.com)

## Prelims

In [1]:
!pip install -q -r requirements-an.txt

You should consider upgrading via the '/Users/wangsuhu/Google Drive/Github/pytorch-CycleGAN-and-pix2pix/env/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import matplotlib.pyplot as plt
import functools
from torch.nn import init
import os
from PIL import Image
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import numpy as np
import itertools
import random
import time
from collections import OrderedDict

## Background

The Cycle Generative Adversarial Network, or CycleGAN, is an approach to training a deep convolutional neural network for image-to-image translation tasks.

Unlike other GAN models for image translation, the CycleGAN does not require a dataset of paired images. 

The CycleGAN model was described by Jun-Yan Zhu, et al. in their 2017 paper titled [“Unpaired Image-to-Image Translation using Cycle-Consistent Adversarial Networks.”](https://arxiv.org/abs/1703.10593)

Here is the official github repo released and maintained by the author list [pytorch-CycleGAN-and-pix2pix](https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix). 

The model architecture is comprised of two generator models: 
- one generator (Generator G_A2B) for generating images for the second class (class-B) based on the input image for the first class (class-A)
- another generator (Generator G_B2A) for generating images for the first class (class-A) based on the input image for the second class (class-B)

> as you can see, unlike normal GAN using randomness, the generator of CycleGAN is conditional on the input image and the input image can be a real image or a generated image. See the generator blocks in workflow below.

    - real image for class-A -> G_A2B -> generated image for class-B 

    - generated image for class-A -> G_A2B -> generated image for class-B 

    - real image for class-B -> G_B2A -> generated image for class-A 

    - generated image for class-B -> G_B2A -> generated image for class-A 

    - real image for class-A -> G_B2A -> generated image for class-A 

    - real image for class-B -> G_A2B -> generated image for class-B 

> the last two seem tricky, it corresponds to identity mapping in CycleGAN which will be explained later.

- one discriminator (Discriminator D_A) takes images from class-A then predicts whether they are real or fake.
- one discriminator (Discriminator D_B) takes images from class-B then predicts whether they are real or fake.

> similar to generators, the input image of discriminator can be a real image or a generated image.

    - real image for class-A -> D_A -> real/fake

    - generated image for class-A -> D_A -> real/fake

    - real image for class-B -> D_B -> real/fake

    - generated image for class-B -> D_B -> real/fake

Why does such GAN model called "Cycle"? 

The discriminator and generator models are trained in an adversarial zero-sum process, like normal GAN models. The generators learn to better fool the discriminators and the discriminator learn to better detect fake images. Together, the models find an equilibrium during the training process.

Additionally, the generator models are regularized to not just create new images in the target domain, but instead translate more reconstructed versions of the input images from the source domain. This is achieved by using generated images as input to the corresponding generator model and comparing the output image to the original images. Passing an image through both generators is called a cycle. Together, each pair of generator models are trained to better reproduce the original source image, referred to as cycle consistency.

    - real image for class-A -> G_A2B -> generated image for class-B -> G_B2A -> generated image for class-A vs the input real image for class-A

    - real image for class-B -> G_B2A -> generated image for class-A -> G_A2B -> generated image for class-B vs the input real image for class-B

Finally, there is identity mapping when a generator model is expected to output an image without translation when provided an example from the target domain. 

    - real image for class-A -> G_B2A -> generated image for class-A

    - real image for class-B -> G_A2B -> generated image for class-B


see the following flow chart for your reference.

![work_flow](workflow.png)

## Prepare Data

We prepare the "horses2zebra" dataset which is a subset imagenet dataset and contains around 111 megabytes and can be downloaded from the CycleGAN webpage:

[Download](https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/horse2zebra.zip)
> after download, put the folder under `annotated_cyclegan/data/`

#### let's first look at what data look like, we randomly selected one image from each class.

In [ ]:
horse_sample = "annotated_cyclegan/data/horse2zebra/testA/n02381460_1000.jpg"
img = plt.imread(horse_sample)
plt.imshow(img)
plt.show()

In [ ]:
zebra_sample = "annotated_cyclegan/data/horse2zebra/testB/n02391049_100.jpg"
img = plt.imread(zebra_sample)
plt.imshow(img)
plt.show()

## Design Dataset Class

Code for propressing data samples can get messy and hard to maintain; we ideally want our dataset code to be decoupled from our model training code for better readability and modularity. Pytorch provides two data primitives: `torch.utils.data.DataLoader` and `torch.utils.data.Dataset` that allow use to use pre-loaded datasets as well as our own data. `Dataset` stores the samples and their corresponding labels; and `DataLoader` wraps an iterable around the the Dataset to enable easy access to the samples. 
> the raw CycleGAN code did the similar thing but in a more customized way. More details can be found under `data/unaligned_dataset.py` and `data/base_dataset.py` and `data/__init__.py`.

As our dataset is not so large (110 MB), we can save them as arrays/vectors then feed into memory later. The normal opt for image pre-processing contains transformation steps such as colorization, scale, crop, and etc. Here we applied what the origin cyclegan paper does, i.e., creating a `UnalignedDataset` which include: 1. load raw image; 2. convert it into tensor; 3. normalize tensor along each color channel; 4. wrap them into a list object. 
> the raw class was defined in `data/unaligned_dataset.py`, here we simplify the code for our own CycleGAN-only experiment.

In [5]:
def get_transform():
    """
    function to do the transformation steps: image -> tensor and normalization.
    """
    transform_list = []
    transform_list += [transforms.ToTensor()]
    transform_list += [transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))] #assume we are dealing with jpg image which contains 3 color channels.
    return transforms.Compose(transform_list)

In [6]:
class UnalignedDataset(torch.utils.data.Dataset):
    """
    This dataset class can load unaligned/unpaired datasets.

    It requires two directories to host training images from domain A '/path/to/data/trainA'
    and from domain B '/path/to/data/trainB' respectively.
    You can train the model with the dataset flag '--dataroot /path/to/data'.
    Similarly, you need to prepare two directories:
    '/path/to/data/testA' and '/path/to/data/testB' during test time.
    """

    def __init__(self, source_class_A_folder, source_class_B_folder):
        """Initialize this dataset class.

        Parameters:
            opt (Option class) -- stores all the experiment flags; needs to be a subclass of BaseOptions
        """
        self.dir_A = source_class_A_folder  # create a path '/path/to/data/trainA'
        self.dir_B = source_class_B_folder  # create a path '/path/to/data/trainB'
        self.transformation = get_transform()
        self.A_images = self.__transform_images__(self.dir_A)
        self.B_images = self.__transform_images__(self.dir_B)
        self.A_size, self.B_size = len(self.A_images), len(self.B_images)
        
        
    def __transform_images__(self, image_folder):
        """Return a transformed images list.

        Parameters:
            image_folder      -- folder contains your images assuming to be of `Image` readable format (such as JPEG, JPG, PNG)

        Returns a list that image tensor
        """
        images = []
        for root, _, fnames in sorted(os.walk(image_folder)):
            for fname in fnames:
                path = os.path.join(root, fname)
                image = Image.open(path).convert('RGB')
                images += [self.transformation(image)]
        return images
    
    
    def __getitem__(self, index):
        """Return a data point and its metadata information.

        Parameters:
            index (int)      -- a random integer for data indexing

        Returns a dictionary that contains A, B
            A (tensor)       -- an image in the input domain
            B (tensor)       -- its corresponding image in the target domain
        """

        return {'A': self.A_images[index % self.A_size], 'B': self.B_images[index % self.B_size]}

    def __len__(self):
        """Return the total number of images in the dataset.

        As we have two datasets with potentially different number of images,
        we take a maximum of
        """
        return max(self.A_size, self.B_size)

In [7]:
train_dataset = UnalignedDataset("annotated_cyclegan/data/horse2zebra/trainA/", "annotated_cyclegan/data/horse2zebra/trainB/")
test_dataset = UnalignedDataset("annotated_cyclegan/data/horse2zebra/testA/", "annotated_cyclegan/data/horse2zebra/testB/")

## Design DataLoader Class

Lets create a basic DataLoader pytorch class.
> CycleGAN do such thing inside the CustomDatasetDataLoader class in `data/__init__.py`

In [8]:
def DataLoader(train_dataset, test_dataset, train_batch, test_batch):
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = train_batch, shuffle = True)
    test_loader = torch.utils.data.DataLoader(train_dataset, batch_size = test_batch, shuffle = False)
    return train_loader, test_loader

In [9]:
train_loader, test_loader = DataLoader(train_dataset, test_dataset, train_batch = 4, test_batch = 1)

## Design Model

As mentioned in section "Background", The architecture is comprised of four models, two discriminator models, and two generator models.

The discriminator is a deep convolutional neural network that performs image classification. It takes a source image as input and predicts the likelihood of whether the target image is a real or fake image. Two discriminator models are used, one for class-A (horses) and one for class-B (zebras).

The discriminator design is based on the effective receptive field of the model, which defines the relationship between one output of the model to the number of pixels in the input image. This is called a PatchGAN model and is carefully designed so that each output prediction of the model maps to a 70×70 square or patch of the input image. The benefit of this approach is that the same model can be applied to input images of different sizes, e.g. larger or smaller than 256×256 pixels.

The output of the model depends on the size of the input image but may be one value or a square activation map of values. Each value is a probability for the likelihood that a patch in the input image is real. These values can be averaged to give an overall likelihood or classification score if needed.

A pattern of Convolutional-BatchNorm-LeakyReLU layers is used in the model, which is common to deep convolutional discriminator models. Unlike other models, the CycleGAN discriminator uses InstanceNormalization instead of BatchNormalization. It is a very simple type of normalization and involves standardizing (e.g. scaling to a standard Gaussian) the values on each output feature map, rather than across features in a batch.

An implementation of instance normalization is provided in the keras-contrib project that provides early access to community supplied Keras features.

> we will use the code from `models/cycle_gan_model.py` and `models/networks.py`.

> the hyperparameter setting is following the default value stored in `options/BaseOptions.py`

### Design discriminator 

Here we assume some hyperparamters as below:

In [10]:
input_nc = 3 # -- the number of channels in input images
ndf = 64 # -- the number of filters in the first conv layer
n_layers_D = 3 # -- the number of conv layers in the discriminator; effective when netD=='n_layers'
norm = "instance" # -- the type of normalization layers used in the network.
init_type = "normal" # -- the name of the initialization method.
init_gain = 0.02 # -- scaling factor for normal, xavier and orthogonal.
netD = "basic" # -- default PatchGAN classifier

In [11]:
class NLayerDiscriminator(nn.Module):
    """Defines a PatchGAN discriminator"""

    def __init__(self, input_nc, ndf=64, n_layers=3, norm_layer=nn.BatchNorm2d):
        """Construct a PatchGAN discriminator

        Parameters:
            input_nc (int)  -- the number of channels in input images
            ndf (int)       -- the number of filters in the last conv layer
            n_layers (int)  -- the number of conv layers in the discriminator
            norm_layer      -- normalization layer
        """
        super(NLayerDiscriminator, self).__init__()
        if type(norm_layer) == functools.partial:  # no need to use bias as BatchNorm2d has affine parameters
            use_bias = norm_layer.func == nn.InstanceNorm2d
        else:
            use_bias = norm_layer == nn.InstanceNorm2d

        kw = 4
        padw = 1
        sequence = [nn.Conv2d(input_nc, ndf, kernel_size=kw, stride=2, padding=padw), nn.LeakyReLU(0.2, True)]
        nf_mult = 1
        nf_mult_prev = 1
        for n in range(1, n_layers):  # gradually increase the number of filters
            nf_mult_prev = nf_mult
            nf_mult = min(2 ** n, 8)
            sequence += [
                nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult, kernel_size=kw, stride=2, padding=padw, bias=use_bias),
                norm_layer(ndf * nf_mult),
                nn.LeakyReLU(0.2, True)
            ]

        nf_mult_prev = nf_mult
        nf_mult = min(2 ** n_layers, 8)
        sequence += [
            nn.Conv2d(ndf * nf_mult_prev, ndf * nf_mult, kernel_size=kw, stride=1, padding=padw, bias=use_bias),
            norm_layer(ndf * nf_mult),
            nn.LeakyReLU(0.2, True)
        ]

        sequence += [nn.Conv2d(ndf * nf_mult, 1, kernel_size=kw, stride=1, padding=padw)]  # output 1 channel prediction map
        self.model = nn.Sequential(*sequence)

    def forward(self, input):
        """Standard forward."""
        return self.model(input)

In [12]:
def init_weights(net):
    """Initialize network weights.

    Parameters:
        net (network)   -- network to be initialized
        init_type (str) -- the name of an initialization method: normal | xavier | kaiming | orthogonal
        init_gain (float)    -- scaling factor for normal, xavier and orthogonal.

    We use 'normal' in the original pix2pix and CycleGAN paper. But xavier and kaiming might
    work better for some applications. Feel free to try yourself.
    """
    def init_func(m):  # define the initialization function
        classname = m.__class__.__name__
        if hasattr(m, 'weight') and (classname.find('Conv') != -1 or classname.find('Linear') != -1):
            init.normal_(m.weight.data, 0.0, init_gain)
            if hasattr(m, 'bias') and m.bias is not None:
                init.constant_(m.bias.data, 0.0)
        elif classname.find('BatchNorm2d') != -1:  # BatchNorm Layer's weight is not a matrix; only normal distribution applies.
            init.normal_(m.weight.data, 1.0, init_gain)
            init.constant_(m.bias.data, 0.0)

    print('initialize network with %s' % init_type)
    net.apply(init_func)  # apply the initialization function <init_func>

In [13]:
def define_D():
    """Create a discriminator 

    Returns a discriminator

    Our current implementation provides three types of discriminators:
        [basic]: 'PatchGAN' classifier described in the original pix2pix paper.
        It can classify whether 70×70 overlapping patches are real or fake.
        Such a patch-level discriminator architecture has fewer parameters
        than a full-image discriminator and can work on arbitrarily-sized images
        in a fully convolutional fashion.

        [n_layers]: With this mode, you can specify the number of conv layers in the discriminator
        with the parameter <n_layers_D> (default=3 as used in [basic] (PatchGAN).)

        [pixel]: 1x1 PixelGAN discriminator can classify whether a pixel is real or not.
        It encourages greater color diversity but has no effect on spatial statistics.

    The discriminator has been initialized by <init_net>. It uses Leakly RELU for non-linearity.
    """
    net = None
    norm_layer = functools.partial(nn.InstanceNorm2d, affine=False, track_running_stats=False) # as we set norm = "instance"
    net = NLayerDiscriminator(input_nc, ndf, n_layers=3, norm_layer=norm_layer) # as we set netD = "basic"
    
    init_weights(net)
    
    return net

In [14]:
D_A = define_D()
D_B = define_D()

initialize network with normal
initialize network with normal


### Design generator 

Here we assume some hyperparamters as below:

In [15]:
input_nc = 3 # -- the number of channels in input images
output_nc = 3 # -- the number of channels in output images
ngf = 64 # -- the number of filters in the first conv layer
n_layers_D = 3 # -- the number of conv layers in the discriminator; effective when netD=='n_layers'
norm = "instance" # -- the type of normalization layers used in the network.
init_type = "normal" # -- the name of the initialization method.
init_gain = 0.02 # -- scaling factor for normal, xavier and orthogonal.
netG = "resnet_9blocks" # -- default CycleGAN generator's architecture
padding_type = 'reflect' # -- the name of padding layer in conv layers

In [16]:
class ResnetBlock(nn.Module):
    """Define a Resnet block"""

    def __init__(self, dim, padding_type, norm_layer, use_dropout, use_bias):
        """Initialize the Resnet block

        A resnet block is a conv block with skip connections
        We construct a conv block with build_conv_block function,
        and implement skip connections in <forward> function.
        Original Resnet paper: https://arxiv.org/pdf/1512.03385.pdf
        """
        super(ResnetBlock, self).__init__()
        self.conv_block = self.build_conv_block(dim, padding_type, norm_layer, use_dropout, use_bias)

    def build_conv_block(self, dim, padding_type, norm_layer, use_dropout, use_bias):
        """Construct a convolutional block.

        Parameters:
            dim (int)           -- the number of channels in the conv layer.
            padding_type (str)  -- the name of padding layer: reflect | replicate | zero
            norm_layer          -- normalization layer
            use_dropout (bool)  -- if use dropout layers.
            use_bias (bool)     -- if the conv layer uses bias or not

        Returns a conv block (with a conv layer, a normalization layer, and a non-linearity layer (ReLU))
        """
        conv_block = []
        p = 0
        conv_block += [nn.ReflectionPad2d(1)] # as we set padding = "reflect"

        conv_block += [nn.Conv2d(dim, dim, kernel_size=3, padding=p, bias=use_bias), norm_layer(dim), nn.ReLU(True)]
        if use_dropout:
            conv_block += [nn.Dropout(0.5)]

        p = 0
        conv_block += [nn.ReflectionPad2d(1)] # as we set padding = "reflect"
        conv_block += [nn.Conv2d(dim, dim, kernel_size=3, padding=p, bias=use_bias), norm_layer(dim)]

        return nn.Sequential(*conv_block)

    def forward(self, x):
        """Forward function (with skip connections)"""
        out = x + self.conv_block(x)  # add skip connections
        return out

In [17]:
class ResnetGenerator(nn.Module):
    """Resnet-based generator that consists of Resnet blocks between a few downsampling/upsampling operations.

    We adapt Torch code and idea from Justin Johnson's neural style transfer project(https://github.com/jcjohnson/fast-neural-style)
    """

    def __init__(self, norm_layer, use_dropout, n_blocks):
        """Construct a Resnet-based generator

        Parameters:
            norm_layer          -- normalization layer
            use_dropout (bool)  -- if use dropout layers
            n_blocks (int)      -- the number of ResNet blocks
        """
        super(ResnetGenerator, self).__init__()
        use_bias = norm_layer.func == nn.InstanceNorm2d # as we set norm = "instance" using functools.partial

        model = [nn.ReflectionPad2d(3),
                 nn.Conv2d(input_nc, ngf, kernel_size=7, padding=0, bias=use_bias),
                 norm_layer(ngf),
                 nn.ReLU(True)]

        n_downsampling = 2
        for i in range(n_downsampling):  # add downsampling layers
            mult = 2 ** i
            model += [nn.Conv2d(ngf * mult, ngf * mult * 2, kernel_size=3, stride=2, padding=1, bias=use_bias),
                      norm_layer(ngf * mult * 2),
                      nn.ReLU(True)]

        mult = 2 ** n_downsampling
        for i in range(n_blocks):       # add ResNet blocks

            model += [ResnetBlock(ngf * mult, padding_type=padding_type, norm_layer=norm_layer, use_dropout=use_dropout, use_bias=use_bias)]

        for i in range(n_downsampling):  # add upsampling layers
            mult = 2 ** (n_downsampling - i)
            model += [nn.ConvTranspose2d(ngf * mult, int(ngf * mult / 2),
                                         kernel_size=3, stride=2,
                                         padding=1, output_padding=1,
                                         bias=use_bias),
                      norm_layer(int(ngf * mult / 2)),
                      nn.ReLU(True)]
        model += [nn.ReflectionPad2d(3)]
        model += [nn.Conv2d(ngf, output_nc, kernel_size=7, padding=0)]
        model += [nn.Tanh()]

        self.model = nn.Sequential(*model)

    def forward(self, input):
        """Standard forward"""
        return self.model(input)

In [18]:
def define_G(use_dropout=True): # for training we will set dropout = True
    """Create a generator

    Parameters:
        use_dropout (bool) -- if use dropout layers.

    Returns a generator

    Our current implementation provides two types of generators:
        U-Net: [unet_128] (for 128x128 input images) and [unet_256] (for 256x256 input images)
        The original U-Net paper: https://arxiv.org/abs/1505.04597

        Resnet-based generator: [resnet_6blocks] (with 6 Resnet blocks) and [resnet_9blocks] (with 9 Resnet blocks)
        Resnet-based generator consists of several Resnet blocks between a few downsampling/upsampling operations.
        We adapt Torch code from Justin Johnson's neural style transfer project (https://github.com/jcjohnson/fast-neural-style).

    The generator has been initialized by <init_net>. It uses RELU for non-linearity.
    """
    net = None
    norm_layer = functools.partial(nn.InstanceNorm2d, affine=False, track_running_stats=False) # as we set norm = "instance"
    net = ResnetGenerator(norm_layer=norm_layer, use_dropout=use_dropout, n_blocks=9) # as we set netG = "resnet_9blocks"
    init_weights(net)
    
    return net

In [19]:
G_A2B = define_G()
G_B2A = define_G()

initialize network with normal
initialize network with normal


### Design CycleGAN model

we first create the basic `nn.Module` torch class of CycleGAN with `__init__()` and `forward()` function

In [20]:
class CycleGANModel(nn.Module):
    def __init__(self):
        super(CycleGANModel, self).__init__()
        self.netG_A2B = G_A2B
        self.netG_B2A = G_B2A
        self.netD_A = D_A
        self.netD_B = D_B
        
    def forward(self, real_A, real_B): # recall the workflow
        self.fake_B = self.netG_A2B(real_A)  # G_A2B(A) # -- generated translated image for class B based on the input real image for class A
        self.pred_fake_B = self.netD_B(self.fake_B) #D_B(G_A2B(A)) -- predict real/fake of the generated image for class B based on the input real image for class A
        self.rec_A = self.netG_B2A(self.fake_B)   # G_A2B(A) # -- generated translated image for class A based on the input generated image for class B
        
        self.fake_A = self.netG_B2A(real_B)  # G_A2B(A) # -- generated translated image for class A based on the input real image for class B
        self.pred_fake_A = self.netD_A(self.fake_A) #D_A(G_B2A(B)) -- predict real/fake of the generated image for class A based on the input real image for class B
        self.rec_B = self.netG_A2B(self.fake_A)   # G_A2B(A) # -- generated translated image for class B based on the input generated image for class A
        
        self.fake_A_ide = self.netG_B2A(real_A)  # G_B2A(A) # -- generated translated image for class A based on the input real image for class A
        self.fake_B_ide = self.netG_A2B(real_B)  # G_A2B(B) # -- generated translated image for class A based on the input real image for class A
        
        return (self.fake_B, self.pred_fake_B, self.rec_A, self.fake_A, self.pred_fake_A, self.rec_B, self.fake_A_ide, self.fake_B_ide) 

In [21]:
model = CycleGANModel()

let us feed a sample input

In [ ]:
image_A_sample_path = "annotated_cyclegan/data/horse2zebra/testA/n02381460_1000.jpg"
image_B_sample_path = "annotated_cyclegan/data/horse2zebra/testB/n02391049_100.jpg"
print("input real image for class A")
plt.imshow(plt.imread(image_A_sample_path))
plt.show()
print("input real image for class B")
plt.imshow(plt.imread(image_B_sample_path))
plt.show()

In [23]:
image_A = Image.open(image_A_sample_path).convert('RGB')
transformed_image_A = get_transform()(image_A)
image_B = Image.open(image_B_sample_path).convert('RGB')
transformed_image_B = get_transform()(image_B)
res = model(transformed_image_A.unsqueeze(0), transformed_image_B.unsqueeze(0))

In [24]:
def tensor2im(input_image, imtype=np.uint8):
    """"Converts a Tensor array into a numpy image array.

    Parameters:
        input_image (tensor) --  the input image tensor array
        imtype (type)        --  the desired type of the converted numpy array
    """
    image_tensor = input_image.data
    image_numpy = image_tensor[0].cpu().float().numpy()
    image_numpy = (np.transpose(image_numpy, (1, 2, 0)) + 1) / 2.0 * 255.0
    return image_numpy.astype(imtype)

let us see the output

In [ ]:
# self.fake_B, self.pred_fake_B, self.rec_A, self.fake_A, self.pred_fake_A, self.rec_B, self.fake_A_ide, self.fake_B_ide
print("generated image for class B based on input real image for class A")
plt.imshow(tensor2im(res[0])) # surely, right now everything is in chaos
plt.show()
print("predict real/fake of generated image for class B based on input real image for class A")
plt.imshow(tensor2im(res[1])) # surely, right now everything is in chaos
plt.show()
print("generated image for class A based on generated image for class B")
plt.imshow(tensor2im(res[2])) # surely, right now everything is in chaos
plt.show()
print("generated image for class A based on input real image for class B")
plt.imshow(tensor2im(res[3])) # surely, right now everything is in chaos
plt.show()
print("predict real/fake of generated image for class A based on input real image for class B")
plt.imshow(tensor2im(res[4])) # surely, right now everything is in chaos
plt.show()
print("generated image for class B based on generated image for class A")
plt.imshow(tensor2im(res[5])) # surely, right now everything is in chaos
plt.show()
print("generated image for class A based on generated image for class A")
plt.imshow(tensor2im(res[6])) # surely, right now everything is in chaos
plt.show()
print("generated image for class B based on generated image for class B")
plt.imshow(tensor2im(res[7])) # surely, right now everything is in chaos
plt.show()

### Design loss function and backward function of the model

The overall loss comes from several components: 
- GAN loss for G_A2B, G_B2A, D_A, D_B; 
- Cycle consistence loss for real_A vs rec_A and real_B vs rec_B; 
- Identity loss for real_A vs fake_A_ide and real_B vs fake_B_ide.

Here we customized the backward function inside the model based on the loss scheme.
> As the paper defines an image pool called `ImagePool` of 50 generated images for each discriminator model that is first populated and probabilistically either adds new images to the pool by replacing an existing image or uses a generated image directly. 

In [26]:
class ImagePool():
    """This class implements an image buffer that stores previously generated images.

    This buffer enables us to update discriminators using a history of generated images
    rather than the ones produced by the latest generators.
    """

    def __init__(self, pool_size):
        """Initialize the ImagePool class

        Parameters:
            pool_size (int) -- the size of image buffer, if pool_size=0, no buffer will be created
        """
        self.pool_size = pool_size
        if self.pool_size > 0:  # create an empty pool
            self.num_imgs = 0
            self.images = []

    def query(self, images):
        """Return an image from the pool.

        Parameters:
            images: the latest generated images from the generator

        Returns images from the buffer.

        By 50/100, the buffer will return input images.
        By 50/100, the buffer will return images previously stored in the buffer,
        and insert the current images to the buffer.
        """
        if self.pool_size == 0:  # if the buffer size is 0, do nothing
            return images
        return_images = []
        for image in images:
            image = torch.unsqueeze(image.data, 0)
            if self.num_imgs < self.pool_size:   # if the buffer is not full; keep inserting current images to the buffer
                self.num_imgs = self.num_imgs + 1
                self.images.append(image)
                return_images.append(image)
            else:
                p = random.uniform(0, 1)
                if p > 0.5:  # by 50% chance, the buffer will return a previously stored image, and insert the current image into the buffer
                    random_id = random.randint(0, self.pool_size - 1)  # randint is inclusive
                    tmp = self.images[random_id].clone()
                    self.images[random_id] = image
                    return_images.append(tmp)
                else:       # by another 50% chance, the buffer will return the current image
                    return_images.append(image)
        return_images = torch.cat(return_images, 0)   # collect all the images and return
        return return_images

In [27]:
class CycleGANModel(nn.Module):
    def __init__(self):
        super(CycleGANModel, self).__init__()
        self.netG_A2B = G_A2B
        self.netG_B2A = G_B2A
        self.netD_A = D_A
        self.netD_B = D_B
        
        self.criterionGAN = torch.nn.MSELoss()
        self.criterionCycle = torch.nn.L1Loss()
        self.criterionIdt = torch.nn.L1Loss()
        
        self.lambda_idt = 0.5 # 'use identity mapping. Setting lambda_identity other than 0 has an effect of scaling the weight of the identity mapping loss. For example, if the weight of the identity loss should be 10 times smaller than the weight of the reconstruction loss, please set lambda_identity = 0.1'
        self.lambda_A = 10.0 # 'weight for cycle loss (B -> A -> B)'
        self.lambda_B = 10.0 # 'weight for cycle loss (A -> B -> A)'
        
        self.lr = 0.0002
        self.beta1 = 0.5
        self.optimizer_G = torch.optim.Adam(itertools.chain(self.netG_A2B.parameters(), self.netG_B2A.parameters()), lr=self.lr, betas=(self.beta1, 0.999))
        self.optimizer_D = torch.optim.Adam(itertools.chain(self.netD_A.parameters(), self.netD_B.parameters()), lr=self.lr, betas=(self.beta1, 0.999))
        self.optimizers = [self.optimizer_G, self.optimizer_D]
        
        self.pool_size = 50
        self.fake_A_pool = ImagePool(self.pool_size)  # create image buffer to store previously generated images
        self.fake_B_pool = ImagePool(self.pool_size)  # create image buffer to store previously generated images
        
        self.loss_names = ['D_A', 'G_A2B', 'cycle_A', 'idt_A', 'D_B', 'G_B2A', 'cycle_B', 'idt_B']
        
    def set_input(self, input):
        """Unpack input data from the dataloader and perform necessary pre-processing steps.

        Parameters:
            input (dict): include the data itself and its metadata information.

        The option 'direction' can be used to swap domain A and domain B.
        """
        self.real_A = input['A']
        self.real_B = input['B']
    
    def get_current_losses(self):
        """Return traning losses / errors. train.py will print out these errors on console, and save them to a file"""
        errors_ret = OrderedDict()
        for name in self.loss_names:
            if isinstance(name, str):
                errors_ret[name] = float(getattr(self, 'loss_' + name))  # float(...) works for both scalar tensor and float number
        return errors_ret
    
    def update_learning_rate(self):
        """Update learning rates for all the networks; called at the end of every epoch"""
        for scheduler in self.schedulers:
            scheduler.step()
        
        
    def forward(self): # recall the workflow
        self.fake_B = self.netG_A2B(self.real_A)  # G_A2B(A) # -- generated translated image for class B based on the input real image for class A
        self.pred_fake_B = self.netD_B(self.fake_B) #D_B(G_A2B(A)) -- predict real/fake of the generated image for class B based on the input real image for class A
        self.rec_A = self.netG_B2A(self.fake_B)   # G_A2B(A) # -- generated translated image for class A based on the input generated image for class B

        self.fake_A = self.netG_B2A(self.real_B)  # G_A2B(A) # -- generated translated image for class A based on the input real image for class B
        self.pred_fake_A = self.netD_A(self.fake_A) #D_A(G_B2A(B)) -- predict real/fake of the generated image for class A based on the input real image for class B
        self.rec_B = self.netG_A2B(self.fake_A)   # G_A2B(A) # -- generated translated image for class B based on the input generated image for class A
        
        self.fake_A_ide = self.netG_B2A(self.real_A)  # G_B2A(A) # -- generated translated image for class A based on the input real image for class A
        self.fake_B_ide = self.netG_A2B(self.real_B)  # G_A2B(B) # -- generated translated image for class A based on the input real image for class A
        
        return (self.fake_B, self.pred_fake_B, self.rec_A, self.fake_A, self.pred_fake_A, self.rec_B, self.fake_A_ide, self.fake_B_ide) 
    
    def backward_D_basic(self, netD, real, fake):
        """Calculate GAN loss for the discriminator

        Parameters:
            netD (network)      -- the discriminator D
            real (tensor array) -- real images
            fake (tensor array) -- images generated by a generator

        Return the discriminator loss.
        We also call loss_D.backward() to calculate the gradients.
        """
        # Real
        pred_real = netD(real)
        real_label_tensor = torch.tensor(1.0).expand_as(pred_real)
        loss_D_real = self.criterionGAN(pred_real, real_label_tensor)
            
        # Fake
        pred_fake = netD(fake)
        fake_label_tensor = torch.tensor(0.0).expand_as(pred_fake)
        loss_D_fake = self.criterionGAN(pred_fake, fake_label_tensor)
        
        # Combined loss and calculate gradients
        loss_D = (loss_D_real + loss_D_fake) * 0.5
        loss_D.backward()
        return loss_D
    
    
    def backward_D_A(self):
        """Calculate GAN loss for discriminator D_A"""
        fake_A = self.fake_A_pool.query(self.fake_A)
        self.loss_D_A = self.backward_D_basic(self.netD_A, self.real_A, fake_A)

    def backward_D_B(self):
        """Calculate GAN loss for discriminator D_B"""
        fake_B = self.fake_B_pool.query(self.fake_B)
        self.loss_D_B = self.backward_D_basic(self.netD_B, self.real_B, fake_B)

    def backward_G(self):
        """Calculate the loss for generators G_A and G_B"""
        
        # Identity loss
        # G_A2B should be identity if real_B is fed: ||G_A2B(B) - B||
        self.loss_idt_A = self.criterionIdt(self.fake_A_ide, self.real_A) * self.lambda_A * self.lambda_idt
        # G_B2A should be identity if real_A is fed: ||G_B2A(A) - A||
        self.loss_idt_B = self.criterionIdt(self.fake_B_ide, self.real_B) * self.lambda_B * self.lambda_idt

        # GAN loss D_B(G_A2B(A))        
        label_tensor_G_A2B = torch.tensor(1.0).expand_as(self.pred_fake_B) # we want the result of D_B(G_A2B(A)) to be True to train the G, aka, to fool D_B
        self.loss_G_A2B = self.criterionGAN(self.pred_fake_B, label_tensor_G_A2B)
        
        # GAN loss D_A(G_B2A(B)) 
        label_tensor_G_B2A = torch.tensor(1.0).expand_as(self.pred_fake_A) # we want the result of D_A(G_B2A(B)) to be True to train the G, aka, to fool D_A
        self.loss_G_B2A = self.criterionGAN(self.pred_fake_A, label_tensor_G_B2A)
                
        # cycle loss || G_B2A(G_A2B(A)) - A||
        self.loss_cycle_A = self.criterionCycle(self.rec_A, self.real_A) * self.lambda_A
        # cycle loss || G_A2B(G_B2A(B)) - B||
        self.loss_cycle_B = self.criterionCycle(self.rec_B, self.real_B) * self.lambda_B
        
        # combined loss and calculate gradients
        self.loss_G = self.loss_G_A2B + self.loss_G_B2A + self.loss_cycle_A + self.loss_cycle_B + self.loss_idt_A + self.loss_idt_B
        self.loss_G.backward()
    
    def set_requires_grad(self, nets, requires_grad=False):
        """Set requies_grad=Fasle for all the networks to avoid unnecessary computations
        Parameters:
            nets (network list)   -- a list of networks
            requires_grad (bool)  -- whether the networks require gradients or not
        """
        if not isinstance(nets, list):
            nets = [nets]
        for net in nets:
            if net is not None:
                for param in net.parameters():
                    param.requires_grad = requires_grad

    def optimize_parameters(self):
        """Calculate losses, gradients, and update network weights; called in every training iteration"""
        # forward
        self.forward()      # compute fake images and reconstruction images.
        # G_A2B and G_B2A
        self.set_requires_grad([self.netD_A, self.netD_B], False)  # Ds require no gradients when optimizing Gs
        self.optimizer_G.zero_grad()  # set G_A and G_B's gradients to zero
        self.backward_G()             # calculate gradients for G_A and G_B
        self.optimizer_G.step()       # update G_A and G_B's weights
        # D_A and D_B
        self.set_requires_grad([self.netD_A, self.netD_B], True)
        self.optimizer_D.zero_grad()   # set D_A and D_B's gradients to zero
        self.backward_D_A()      # calculate gradients for D_A
        self.backward_D_B()      # calculate graidents for D_B
        self.optimizer_D.step()  # update D_A and D_B's weights

### model training

we followed the alternating training approach to update the parameters of generators and discriminator. As mentioned in Google's GAN training [documents](https://developers.google.com/machine-learning/gan/training):

```
The generator and the discriminator have different training processes. So how do we train the GAN as a whole?

GAN training proceeds in alternating periods:

The discriminator trains for one or more epochs.
The generator trains for one or more epochs.
Repeat steps 1 and 2 to continue to train the generator and discriminator networks.
We keep the generator constant during the discriminator training phase. As discriminator training tries to figure out how to distinguish real data from fake, it has to learn how to recognize the generator's flaws. That's a different problem for a thoroughly trained generator than it is for an untrained generator that produces random output.

Similarly, we keep the discriminator constant during the generator training phase. Otherwise the generator would be trying to hit a moving target and might never converge.

It's this back and forth that allows GANs to tackle otherwise intractable generative problems. We get a toehold in the difficult generative problem by starting with a much simpler classification problem. Conversely, if you can't train a classifier to tell the difference between real and generated data even for the initial random generator output, you can't get the GAN training started.

Convergence

As the generator improves with training, the discriminator performance gets worse because the discriminator can't easily tell the difference between real and fake. If the generator succeeds perfectly, then the discriminator has a 50% accuracy. In effect, the discriminator flips a coin to make its prediction.

This progression poses a problem for convergence of the GAN as a whole: the discriminator feedback gets less meaningful over time. If the GAN continues training past the point when the discriminator is giving completely random feedback, then the generator starts to train on junk feedback, and its own quality may collapse.

For a GAN, convergence is often a fleeting, rather than stable, state.
```


In [31]:
def visualize(model):
    fig, axs = plt.subplots(1,8, figsize = (40, 5))
    axs[0].imshow(tensor2im(model.real_A[0:1]))
    # plt.show()
    axs[1].imshow(tensor2im(model.fake_B[0:1]))
    # plt.show()
    axs[2].imshow(tensor2im(model.rec_A[0:1]))
    # plt.show()
    axs[3].imshow(tensor2im(model.fake_A_ide[0:1]))
    # plt.show()
    axs[4].imshow(tensor2im(model.real_B[0:1]))
    # plt.show()
    axs[5].imshow(tensor2im(model.fake_A[0:1]))
    # plt.show()
    axs[6].imshow(tensor2im(model.rec_B[0:1]))
    # plt.show()
    axs[7].imshow(tensor2im(model.fake_B_ide[0:1]))
    plt.show()

In [ ]:
max_epoch = 100
model = CycleGANModel()
for epoch in range(max_epoch):    
    print(f"epoch: {epoch}")
    for i, data in enumerate(train_loader):  # inner loop within one epoch
        model.set_input(data)         # unpack data from dataset and apply preprocessing
        model.optimize_parameters()   # calculate loss functions, get gradients, update network weights
        losses = model.get_current_losses()
        print(f"step: {i}, losses: {losses}")
        print(f"D_A mean prob: {model.pred_fake_A.mean()}; D_B mean prob: {model.pred_fake_B.mean()}")
        visualize(model)